In [1]:
!nvidia-smi

Thu Jun  9 08:45:19 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    23W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import json
import re
import string

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# # Install the vncorenlp python wrapper
# !pip3 install vncorenlp

# # Download VnCoreNLP-1.1.1.jar & its word segmentation component (i.e. RDRSegmenter) 
# !mkdir -p vncorenlp/models/wordsegmenter
# !wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
# !wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
# !wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr
# !mv VnCoreNLP-1.1.1.jar vncorenlp/
# !mv vi-vocab vncorenlp/models/wordsegmenter/
# !mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/

In [4]:
from vncorenlp import VnCoreNLP
rdrsegmenter = VnCoreNLP("/content/vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m') 

In [5]:
# Input 
text = "Ông Nguyễn Khắc Chúc  đang làm việc tại Đại học Quốc gia Hà Nội. Bà Lan, vợ ông Chúc, cũng làm việc tại đây."

# To perform word (and sentence) segmentation
sentences = rdrsegmenter.tokenize(text) 
for sentence in sentences:
	print(" ".join(sentence))

Ông Nguyễn_Khắc_Chúc đang làm_việc tại Đại_học Quốc_gia Hà_Nội .
Bà Lan , vợ ông Chúc , cũng làm_việc tại đây .


# Legal Corpus

## Read data

In [6]:
with open('/content/drive/MyDrive/NLP/Information_Retrieval/Legal_Text_Retrieval_Zalo2021/datasets/legal_corpus.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

In [7]:
legal_corpus_dfs = []

for i in range(len(data)):
    corpus_tmp_df = pd.DataFrame.from_dict(data[i]["articles"])
    corpus_tmp_df['law_id'] = data[i]['law_id']
    legal_corpus_dfs.append(corpus_tmp_df)

legal_corpus_df = pd.concat(legal_corpus_dfs).set_index(['law_id', 'article_id'])

## EDA Legal Corpus

In [8]:
legal_corpus_df['text'] = legal_corpus_df['title'] + '. ' + legal_corpus_df['text']
legal_corpus_df.drop('title', inplace=True, axis=1)
# legal_corpus_df['text_word_count'] = legal_corpus_df['text'].apply(lambda x: len(x.split(' ')))

In [9]:
legal_corpus_df

text
law_id         article_id                                                   
01/2009/tt-bnn 1           Điều 1. Phạm vi áp dụng. Thông tư này hướng dẫ...
               2           Điều 2. Tổ chức lực lượng. 1. Hàng năm trước m...
               3           Điều 3. Tiêu chuẩn của các thành viên thuộc lự...
               4           Điều 4. Nhiệm vụ của lực lượng tuần tra, canh ...
               5           Điều 5. Phù hiệu của lực lượng tuần tra, canh ...
...                                                                      ...
99/2020/nđ-cp  60          Điều 60. Thẩm quyền xử phạt của Hải quan. 1. C...
               61          Điều 61. Thẩm quyền xử phạt của Quản lý thị tr...
               62          Điều 62. Thẩm quyền xử phạt của Thanh tra. 1. ...
               63          Điều 63. Phân định thẩm quyền xử phạt của Chủ ...
               64          Điều 64. Hiệu lực thi hành. 1. Nghị định này c...

[61425 rows x 1 columns]

In [10]:
number = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10"]
chars = ["a", "b", "c", "d", "đ", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o"]
stop_word = number + chars + ["của", "và", "các", "có", "được", "theo", "tại", "trong", "về", 
                              "hoặc", "người",  "này", "khoản", "cho", "không", "từ", "phải", 
                              "ngày", "việc", "sau",  "để",  "đến", "bộ",  "với", "là", "năm", 
                              "khi", "số", "trên", "khác", "đã", "thì", "thuộc", "điểm", "đồng",
                              "do", "một", "bị", "vào", "lại", "ở", "nếu", "làm", "đây", 
                              "như", "đó", "mà", "nơi", "điều"]

def preprocess(text):
    # Converting to lowercase
    text = text.lower()

    # Remove \n \t
    text = re.sub(r'\n+', ' ', text)
    text = re.sub(r'\t+', ' ', text)

    # Remove numbers
    text = re.sub(r'[0-9]+', ' ', text)

    # Remove punctuation
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)

    # Remove ""
    text = re.sub(r'(”|“)', ' ', text)

    # Remove multiple spaces
    text = re.sub(r'\s+', ' ', text)

    try:
        sentences = rdrsegmenter.tokenize(text)
        text = ""
        for sentence in sentences:
            text += " ".join(sentence)
    except:
        print("Can not tokenizer: ", text)
    
    # Remove stop words
    words = text.split(' ')
    words = [word for word in words if word not in stop_word]
    text = " ".join(words)

    return text


def preprocess_df(df):
    """
    This function is for processing source of notebook
    returns preprocessed dataframe
    """
    df['text'] = df['text'].apply(preprocess)
    return df

In [11]:
clean_legal_corpus_df = preprocess_df(legal_corpus_df)

Can not tokenizer:  điều sửa đổi bổ sung bãi bỏ một số điều của bộ luật hình sự số qh sửa đổi bổ sung điều như sau a sửa đổi bổ sung điểm d khoản điều như sau d nghiêm trị người phạm tội dùng thủ đoạn xảo quyệt có tổ chức có tính chất chuyên nghiệp cố ý gây hậu quả đặc biệt nghiêm trọng khoan hồng đối với người tự thú đầu thú thành khẩn khai báo tố giác đồng phạm lập công chuộc tội tích cực hợp tác với cơ quan có trách nhiệm trong việc phát hiện tội phạm hoặc trong quá trình giải quyết vụ án ăn năn hối cải tự nguyện sửa chữa hoặc bồi thường thiệt hại gây ra b sửa đổi bổ sung điểm d khoản điều như sau d khoan hồng đối với pháp nhân thương mại tích cực hợp tác với cơ quan có trách nhiệm trong việc phát hiện tội phạm hoặc trong quá trình giải quyết vụ án tự nguyện sửa chữa bồi thường thiệt hại gây ra chủ động ngăn chặn hoặc khắc phục hậu quả xảy ra sửa đổi bổ sung điều như sau điều phân loại tội phạm căn cứ vào tính chất và mức độ nguy hiểm cho xã hội của hành vi phạm tội được quy định tr

In [13]:
clean_legal_corpus_df['text'][0]

'phạm_vi áp_dụng thông_tư hướng_dẫn tuần_tra canh_gác bảo_vệ đê_điều mùa lũ đối_với tuyến đê sông phân_loại phân_cấp quy_định luật đê_điều'

In [14]:
clean_legal_corpus_df.to_pickle('/content/drive/MyDrive/NLP/Information_Retrieval/Legal_Text_Retrieval_Zalo2021/datasets/clean_legal_corpus.pkl')